In [1]:
# import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

True

# 定义模型

获取api_key

https://open.bigmodel.cn/dev/api/normal-model/glm-4#auth

In [20]:
from langchain_openai import ChatOpenAI

api_key = os.getenv("zhipu_api_key")

model = ChatOpenAI(
    model = 'glm-4-flash',
    openai_api_key=api_key,
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
)


In [21]:
import os
from langchain_openai import ChatOpenAI
api_key = os.getenv("zhipu_api_key")
# print(api_key)
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4-flash",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)



In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
[
    ("system", "你是世界级的技术专家"),
    ("user", "{input}"),
]
)



In [23]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser() # 字符解析器
chain = prompt|llm |output_parser  
result = chain.invoke({"input":"帮我写一篇关于AI的技术文章，100个字"})
print(result)

# 如果不使用StrOutputParser的话输出的是一个对象，如果的话会输出一个字符串


人工智能技术正飞速发展，从语音识别到自动驾驶，AI正改变着我们的生活。随着算法和硬件的进步，AI将在更多领域发挥巨大作用，推动社会进步。


# 定义工具

## 搜索工具

In [24]:
from langchain_community.tools import DuckDuckGoSearchRun

## 数据库操作工具

In [25]:
import pandas as pd
from pandasql import sqldf

def simulate_database_operation(sql):
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result
    

In [26]:
print(simulate_database_operation('SELECT * FROM my_table WHERE age > 50'))


              name  age sex
0      Henry Myers   60   F
1      Kelsey Lutz   54   M
2   Jonathan Young   76   F
3  Kimberly Macias   69   M


In [27]:
print(simulate_database_operation('SELECT * FROM my_table WHERE id > 50'))


PandaSQLException: (sqlite3.OperationalError) no such column: id
[SQL: SELECT * FROM my_table WHERE id > 50]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [28]:
from langchain.tools import tool


@tool
def simulate_database_operation(sql: str):
	'''根据sql语句操作数据库'''
	my_table = pd.DataFrame({
	    'name': ['Henry Myers', 'Martha Hawkins', 'Kelsey Lutz', 'Jonathan Fowler', 'Jonathan Young', 'Autumn Johnson', 'Kimberly Macias', 'Jared Mccormick', 'Casey Hoover', 'Erica Morse'],
	    'age': [60, 44, 54, 46, 76, 22, 69, 33, 23, 35],
	    'sex': ['F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'F', 'M']
	})
	result = sqldf(sql)
	return result



## 模型调用工具

In [29]:
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
model_with_tools = model.bind_tools(tools)



In [30]:
from pprint import pprint as pp

response = model_with_tools.invoke('印度的首都是哪里？')
pp(dict(response))


{'additional_kwargs': {'refusal': None},
 'content': '印度的首都是新德里（New '
            'Delhi）。它是印度联邦的首都，也是德里联邦属地的首府。新德里不仅是印度的政治中心，也是重要的文化、教育、商业和交通枢纽。',
 'example': False,
 'id': 'run-cd3d78f9-73d4-40c8-9c06-ca2bcfc86f49-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': '20250412181834cf68421d152546de',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 43,
                                       'completion_tokens_details': None,
                                       'prompt_tokens': 275,
                                       'prompt_tokens_details': None,
                                       'total_tokens': 318}},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {'input_token_details': {},
                    'input_tokens': 275,
                    'output_to

In [31]:
fetch_response = model_with_tools.invoke('现在是北京时间几点？查询之后告诉我')
pp(dict(fetch_response))


{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query": '
                                                                 '"北京时间现在几点"}',
                                                    'name': 'duckduckgo_search'},
                                       'id': 'call_-8802642330699537916',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-9d5b9e29-d1cf-4922-adaa-14ad405e0990-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': '20250412181947895b5a3cda614188',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 14,
                                       'completion_tokens_details': None,
    

In [33]:
fetch_response = model_with_tools.invoke('今天有什么新闻？请搜索后告诉我')
pp(dict(fetch_response))

{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query": '
                                                                 '"今天新闻"}',
                                                    'name': 'duckduckgo_search'},
                                       'id': 'call_-8802672429831723648',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-df725eaa-2bd9-4e71-8f94-1d6502ddef52-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': '20250412182227b9cdcef133c34c8c',
                       'logprobs': None,
                       'model_name': 'glm-4-flash',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 13,
                                       'completion_tokens_details': None,
        

In [32]:
db_response = model_with_tools.invoke('帮我往数据库的my_table表中插入一条数据，name是张三，age是18，sex是male')
pp(dict(db_response))


{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"sql": '
                                                                 '"INSERT INTO '
                                                                 'my_table '
                                                                 '(name, age, '
                                                                 'sex) VALUES '
                                                                 "('张三', 18, "
                                                                 '\'male\')"}',
                                                    'name': 'simulate_database_operation'},
                                       'id': 'call_-8802641299907141527',
                                       'index': 0,
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run-10b6db1c-cdfd-4ede-a2ee-ad411d5f2819-0',
 'invalid_tool_calls': [],
 'name': None

# 定义agent

## 手动定义agent

In [39]:
import icecream as ic


def manual_agent(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(
        f'original query: {query} \n\n\n tool response: {tool_response}'
    )
    return final_response


def call_tool(model_output, tools):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool['name']
        tool_args = tool['args']
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response



In [40]:
manual_agent('帮我查询数据库my_table表中有多少人年龄大于60', model, tools).content


'根据工具的响应，查询结果显示在数据库`my_table`表中，年龄大于60岁的人有2人。工具使用了`COUNT(*)`函数来计算符合条件的记录数，返回的结果是0和2，其中0可能是查询语句执行前的默认值或占位符，而2是实际的查询结果。因此，正确答案是`my_table`表中有2人的年龄大于60岁。'

## 使用langgraph定义agent

In [41]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(model, tools)

In [ ]:
from langchain_core.messages import HumanMessage
response = agent.invoke({'messages': [HumanMessage(content="今天在北京的天气怎么样")]})
response
